In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

In [ ]:
# Get Data: Do not touch it.
def get_data():
  data_url = "http://lib.stat.cmu.edu/datasets/boston"
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
  X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
  y = raw_df.values[1::2, 2]
  return X,y

X,y  = get_data()

X.shape

(506, 13)

In [ ]:
# cgs
def cgs(A):
  """
    Q,R = cgs(A)
    Apply classical Gram-Schmidt to mxn rectangular/square matrix. 

    Parameters
    -------
    A: mxn rectangular/square matrix   

    Returns
    -------
    Q: mxn square matrix
    R: nxn upper triangular matrix

  """
  m = len(A)
  n = len(A[0])
  R = np.zeros((n,n))
  Q = np.ones((m,n))
  for k in range(n):
    w = A[:,k]
    for j in range(1,k-1):
      R[j,k] = np.dot(Q[:,j],w)
    for j in range(k):
        w = w - R[j,k]*Q[:,j]
    R[k,k] = np.linalg.norm(w)
    Q[:,k] = w/R[k,k]

  return R,Q

In [ ]:
# Implement BACK SUBS
def backsubs(U, b):

  """
  x = backsubs(U, b)
  Apply back substitution for the square upper triangular system Ux=b. 

  Parameters
  -------
    U: nxn square upper triangular array
    b: n array
    

  Returns
  -------
    x: n array
  """

  n= U.shape[1]
  x= np.zeros((n,))
  b_copy= np.copy(b)

  if U[n-1,n-1]==0.0:
    if b[n-1] != 0.0:
      print("System has no solution.")
  
  else:
    x[n-1]= b_copy[n-1]/U[n-1,n-1]
  for i in range(n-2,-1,-1):
    if U[i,i]==0.0:
      if b[i]!= 0.0:
        print("System has no solution.")
    else:
      for j in range(i,n):
        b_copy[i] -=U[i,j]*x[j]
      x[i]= b_copy[i]/U[i,i]
  return x

In [ ]:
def cgss(A,Y):
  R,Q = cgs(A)
  y = Q.T@Y
  y_ = backsubs(R,y)
  return y_
cgss(X,y)

array([ 5.84191562e-01,  3.91360483e-01,  2.13702193e+00,  5.23063143e+01,
        3.57578406e+01,  3.64287777e-01, -1.59843432e-01,  1.19358393e+00,
       -4.68938580e-01, -1.29304652e-02, -3.50212442e-01, -4.06736554e-03,
       -4.89167210e-01])

In [ ]:
# Add ones
def add_ones(w):

  one = np.ones((len(w))).reshape(len(w),1)

  w = np.hstack((one,w))
  return w

In [ ]:
## Add ones to X
X= add_ones(X)
X

array([[1.0000e+00, 6.3200e-03, 1.8000e+01, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [1.0000e+00, 2.7310e-02, 0.0000e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [1.0000e+00, 2.7290e-02, 0.0000e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [1.0000e+00, 6.0760e-02, 0.0000e+00, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0000e+00, 1.0959e-01, 0.0000e+00, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [1.0000e+00, 4.7410e-02, 0.0000e+00, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [ ]:
def split_data(X,Y, train_size):
  # ADD YOUR CODES
  # shuffle the data before splitting it
  x_train_size = round(len(X)* train_size)
  x_test_size = len(X) - x_train_size
  
  np.random.shuffle(X)
  np.random.shuffle(Y)
  X_train = X[:x_train_size]
  X_test = X[x_train_size:]
  y_train = y[:x_train_size]
  y_test = y[x_train_size:]


  return X_train, X_test, y_train, y_test

In [ ]:
# Split (X,y) into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test= split_data(X,y,0.8)

In [ ]:
def mse(y, y_pred):
  error = y- y_pred
  ms = np.dot(error,error)/len(y)
  return ms

In [ ]:
def normalEquation(X,y):
    # ADD YOUR CODES

    theta_hat = np.linalg.inv(np.transpose(X) @ X) @ (np.transpose(X)@y)
    return theta_hat

In [ ]:
b = normalEquation(X_train,y_train)
print(b)

[ 1.70917234e+01  1.15456625e-02 -4.35696390e-02 -9.86898541e-02
  3.34112058e-02 -1.81366437e+00  8.96741262e-01  3.91764445e-02
  1.04537631e+00  1.55286815e-01 -3.26407263e-03 -1.78218969e-01
 -9.56272036e-03  1.73957033e-01]


In [ ]:

def predict(x,a):
  y = x@a
  return y

Y = predict(X_test,b)

u = mse(y_test,Y)
u


91.22030142819699

In [ ]:
class LinearRegression:

  def __init__(self, model= "lin"):
      # ADD YOUR CODES
     self.model = model
  def fit(self,x,y):
      # ADD YOUR CODES
      self.x = x
      self.y = y
      if self.model == "lin":
        self.theta = normalEquation(self.x,self.y)
      elif self.model == "cgs":
        self.theta = cgss(self.x,self.y)
      else:
        return "Unknown estimator"
        
        #norma
    
  def predict(self,x):
      #ADD YOUR CODES
      y_predict = x @ self.theta

      return y_predict

In [ ]:
# Instanciate the LinearRegression class 
model= LinearRegression("cgs")

model1 = LinearRegression()


In [ ]:
# Train the model

model.fit(X_train, y_train)
model1.fit(X_train, y_train)

In [ ]:
# print the learned theta

print(model.theta)
print(model1.theta)

[ 2.24212346e+01  9.01543031e-02  1.12381185e-01  9.04527879e-01
  1.92177663e+01  1.76059108e+01  2.67717497e-01 -4.18949809e-02
  1.46088653e+00 -1.69700726e-01 -3.70537475e-03 -1.95248350e-02
 -4.03203970e-03  1.70192736e-02]
[ 1.70917234e+01  1.15456625e-02 -4.35696390e-02 -9.86898541e-02
  3.34112058e-02 -1.81366437e+00  8.96741262e-01  3.91764445e-02
  1.04537631e+00  1.55286815e-01 -3.26407263e-03 -1.78218969e-01
 -9.56272036e-03  1.73957033e-01]


In [ ]:
# Make a prediction on X_test

y_pp = model.predict(X_test)
y_pp1 = model1.predict(X_test)

print(y_pp)


print(y_pp1)

[47.91948214 46.33481003 43.7018969  37.04425125 51.41807394 45.14515661
 40.07978684 39.37490033 53.55727566 36.80851902 44.91691219 42.78992461
 39.85575324 45.16891796 33.89084885 33.42188864 45.19893425 36.21457843
 43.44995304 42.9118814  39.63927736 36.90077868 61.1117215  37.42703975
 43.35567351 35.93618094 47.76751334 37.73324336 44.48854642 39.41061125
 43.78945235 40.04479277 44.71170885 45.71517685 34.01788326 41.74448182
 43.93304527 35.81989445 43.54475074 43.94187099 37.79365662 44.022668
 40.28003389 50.22809901 48.35439673 34.1406831  48.48413881 43.0150723
 41.40199821 64.90166084 37.88294108 71.3206483  47.86399913 49.40512403
 47.88646999 46.74481518 44.75662521 46.63422823 41.8393816  41.9161892
 44.92892145 43.96046533 43.91368354 36.69599356 35.2915463  44.5360765
 54.42473061 43.41934301 49.41673293 39.48060969 43.14210301 43.67005793
 35.17222642 43.89627193 46.41367038 62.29377478 46.18507488 53.71222413
 46.60460061 52.01720796 40.98215178 37.96308719 45.5218

In [ ]:
# Compute the MSE (Evaluate both, regression and classification)

normal = mse(y_test,y_pp)
print(normal)

normal1 = mse(y_test,y_pp1)

print(normal1)

570.15914357453
91.22030142819699
